# Starter Notebook

Install and import required libraries

In [1]:
# !pip install transformers datasets evaluate accelerate peft trl bitsandbytes
# !pip install nvidia-ml-py3

In [2]:
import os
import pandas as pd
import torch
import random
import re
import nltk
import pickle
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.char as nac

from transformers import (
    RobertaModel,
    RobertaTokenizer, 
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    RobertaForSequenceClassification,
    EarlyStoppingCallback
)
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import load_dataset, Dataset, ClassLabel

# Download required NLTK data
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/viewsetting/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/viewsetting/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
# 设置设备
device = torch.device('cuda:1' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Using Device: {device}")

Using Device: cuda:1


In [4]:
import wandb
wandb.login(key="2008ab8d896bfc68619ace7f820e0513468b9783", relogin=True)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/viewsetting/.netrc


True

In [5]:
# Get current wandb entity
current_entity = wandb.api.default_entity
print(f"Current wandb entity: {current_entity}")


Current wandb entity: jl10897-new-york-university


## Load Tokenizer and Preprocess Data

In [6]:
# base_model = 'roberta-base'
# # base_model = 'roberta-large'

# dataset = load_dataset('ag_news', split='train')
# tokenizer = RobertaTokenizer.from_pretrained(base_model)

# def clean_text(text):
#     text = re.sub(r"<.*?>", "", text)                 # 去除 HTML 标签
#     text = re.sub(r"http\S+|www\S+", "", text)        # 移除 URL
#     text = re.sub(r"[^A-Za-z0-9.,!?;:'\"()\[\]\s]", "", text)  # 去除特殊字符
#     text = re.sub(r"\s+", " ", text).strip()          # 去除多余空格
#     return text.lower()                               # 可选：统一小写
    
# def preprocess(examples):
#     cleaned_texts = [clean_text(t) for t in examples['text']]
#     return tokenizer(cleaned_texts, truncation=True, padding=True, max_length=256)

# tokenized_dataset = dataset.map(preprocess, batched=True,  remove_columns=["text"])
# tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

In [7]:
nltk.download('averaged_perceptron_tagger_eng', download_dir='/home/viewsetting/nltk_data')



[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/viewsetting/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [8]:
nltk.data.path.append('/home/viewsetting/nltk_data')

In [9]:
import numpy as np
base_model = 'roberta-base'
# base_model = 'roberta-large'

dataset = load_dataset('ag_news', split='train')
tokenizer = RobertaTokenizer.from_pretrained(base_model)

# 初始化增强器
synonym_aug = naw.SynonymAug(aug_src='wordnet', aug_p=0.3)
delete_aug = naw.RandomWordAug(action='delete', aug_p=0.1)
swap_aug = naw.RandomWordAug(action='swap', aug_p=0.1)
typo_aug = nac.RandomCharAug(action='swap', aug_char_p=0.05, aug_word_p=0.1)

# 定义情感词列表（按类别）
sentiment_words = {
    'World': ['terrible', 'urgent', 'critical', 'important', 'serious'],
    'Sports': ['exciting', 'thrilling', 'great', 'amazing', 'intense'],
    'Business': ['profitable', 'successful', 'promising', 'risky', 'innovative'],
    'Sci/Tech': ['advanced', 'innovative', 'futuristic', 'complex', 'technical']
}

def sentiment_word_insertion(text, label, aug_p=0.3):
    """随机插入情感词增强"""
    words = text.split()
    if random.random() < aug_p:
        label_name = ['World', 'Sports', 'Business', 'Sci/Tech'][label]
        sentiment_list = sentiment_words[label_name]
        sentiment_word = random.choice(sentiment_list)
        insert_pos = random.randint(0, len(words))
        words.insert(insert_pos, sentiment_word)
    return ' '.join(words)

def clean_text(text):
    """清理文本：去除HTML标签、URL、特殊字符，并统一小写"""
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"[^A-Za-z0-9.,!?;:'\"()\[\]\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.lower()

def simple_paraphrase(text):
    """简单基于规则的释义"""
    replacements = {
        'said': 'stated',
        'big': 'large',
        'small': 'tiny',
        'good': 'excellent',
        'bad': 'poor',
        'buy': 'purchase',
        'sell': 'trade',
        'make': 'create',
        'show': 'display',
        'start': 'begin'
    }
    words = text.split()
    for i, word in enumerate(words):
        if word.lower() in replacements and random.random() < 0.3:
            words[i] = replacements[word.lower()]
    return ' '.join(words)

def process_single_example(example, augmentations=None):
    """处理单个样本：清理文本、应用增强、分词"""
    if augmentations is None:
        augmentations = {
            'synonym': False,
            'delete': False,
            'swap': False,
            'paraphrase': False,
            'noise': False,
            'sentiment': False
        }

    # 检查是否是批处理模式
    if isinstance(example['text'], (list, np.ndarray)):
        # 批处理模式
        texts = example['text']
        labels = example['label']
        
        # 处理每个文本
        cleaned_texts = [clean_text(t) for t in texts]
        aug_texts = []
        
        for text, label in zip(cleaned_texts, labels):
            aug_text = text
            if augmentations.get('sentiment', False):
                aug_text = sentiment_word_insertion(aug_text, label, aug_p=0.3)
            if augmentations.get('synonym', False):
                aug_text = synonym_aug.augment(aug_text)[0]
            if augmentations.get('delete', False):
                aug_text = delete_aug.augment(aug_text)[0]
            if augmentations.get('swap', False):
                aug_text = swap_aug.augment(aug_text)[0]
            if augmentations.get('paraphrase', False):
                aug_text = simple_paraphrase(aug_text)
            if augmentations.get('noise', False):
                aug_text = typo_aug.augment(aug_text)[0]
            aug_texts.append(aug_text)
            
        # 批量分词
        return tokenizer(aug_texts, truncation=True, padding=True, max_length=256)
    else:
        # 单个样本模式
        text = clean_text(example['text'])
        label = example['label'] if isinstance(example, dict) else example.label

        # 应用增强
        aug_text = text
        if augmentations.get('sentiment', False):
            aug_text = sentiment_word_insertion(aug_text, label, aug_p=0.3)
        if augmentations.get('synonym', False):
            aug_text = synonym_aug.augment(aug_text)[0]
        if augmentations.get('delete', False):
            aug_text = delete_aug.augment(aug_text)[0]
        if augmentations.get('swap', False):
            aug_text = swap_aug.augment(aug_text)[0]
        if augmentations.get('paraphrase', False):
            aug_text = simple_paraphrase(aug_text)
        if augmentations.get('noise', False):
            aug_text = typo_aug.augment(aug_text)[0]

        # 分词
        tokenized = tokenizer(aug_text, truncation=True, padding=True, max_length=256)
        tokenized['labels'] = label
        return tokenized
    
def preprocess_dataset(dataset, augmentations=None, use_parallel=False, num_workers=None):
    """
    预处理数据集，支持并行和非并行处理
    
    Args:
        dataset: 要处理的数据集
        augmentations: 数据增强配置
        use_parallel: 是否使用并行处理
        num_workers: 并行处理的工作进程数
    """
    if use_parallel:
        # 并行处理
        os.environ["TOKENIZERS_PARALLELISM"] = "false"
        if num_workers is None:
            num_workers = max(1, cpu_count() - 1)
        
        data_list = [example for example in dataset]
        process_func = partial(process_single_example, augmentations=augmentations)
        
        with Pool(num_workers) as pool:
            tokenized_examples = pool.map(process_func, data_list)
            
        tokenized_dataset = Dataset.from_dict({
            'input_ids': [ex['input_ids'] for ex in tokenized_examples],
            'attention_mask': [ex['attention_mask'] for ex in tokenized_examples],
            'labels': [ex['labels'] for ex in tokenized_examples]
        })
    else:
        # 非并行处理
        tokenized_dataset = dataset.map(
            lambda examples: process_single_example(examples, augmentations),
            batched=True,
            remove_columns=["text"]
        )
    
    return tokenized_dataset

# 示例增强配置
augmentation_config = {
    'synonym': True,
    'delete': True,
    'swap': True,
    'paraphrase': True,
    'noise': True,
    'sentiment': True  # 启用情感词插入
}

# 使用时可以选择是否启用并行处理
import time
start_time = time.time()
tokenized_dataset = preprocess_dataset(
    dataset, 
    augmentations=augmentation_config,
    use_parallel=False,  # 设置为 False 则使用非并行处理
    num_workers=8
)


/cache_intel/anaconda/miniconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Parameter 'function'=<function preprocess_dataset.<locals>.<lambda> at 0x700964cf34c0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

In [10]:
# Extract the number of classess and their names
num_labels = dataset.features['label'].num_classes
class_names = dataset.features["label"].names
print(f"number of labels: {num_labels}")
print(f"the labels: {class_names}")

# Create an id2label mapping
# We will need this for our classifier.
id2label = {i: label for i, label in enumerate(class_names)}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")


number of labels: 4
the labels: ['World', 'Sports', 'Business', 'Sci/Tech']


## Load Pre-trained Model
Set up config for pretrained model and download it from hugging face

In [11]:
model = RobertaForSequenceClassification.from_pretrained(
    base_model,
    id2label=id2label)
model

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

## Anything from here on can be modified

In [12]:
# Split the original training set
split_datasets = tokenized_dataset.train_test_split(test_size=640, seed=42)
train_dataset = split_datasets['train']
eval_dataset = split_datasets['test']

## Setup LoRA Config
Setup PEFT config and get peft model for finetuning

In [13]:
peft_config = LoraConfig(
    r=4,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules=["query", "value"],
    task_type="SEQ_CLS"
)

In [14]:
peft_model = get_peft_model(model, peft_config)
peft_model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): Module

In [15]:
print('PEFT Model')
peft_model.print_trainable_parameters()

PEFT Model
trainable params: 741,124 || all params: 125,389,832 || trainable%: 0.5911


## Training Setup

In [19]:
import wandb
import seaborn as sns
import matplotlib.pyplot as plt
from typing import Dict
from sklearn.metrics import (
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score, 
    confusion_matrix,
    classification_report
)

def compute_metrics(pred):
    """
    Compute and log comprehensive evaluation metrics
    """
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    probs = torch.nn.functional.softmax(torch.tensor(pred.predictions), dim=-1)
    
    # Basic metrics
    accuracy = accuracy_score(labels, preds)
    precision_macro = precision_score(labels, preds, average='macro')
    precision_micro = precision_score(labels, preds, average='micro')
    precision_per_class = precision_score(labels, preds, average=None)
    
    recall_macro = recall_score(labels, preds, average='macro')
    recall_micro = recall_score(labels, preds, average='micro')
    recall_per_class = recall_score(labels, preds, average=None)
    
    f1_macro = f1_score(labels, preds, average='macro')
    f1_micro = f1_score(labels, preds, average='micro')
    f1_per_class = f1_score(labels, preds, average=None)
    
    # Per-class metrics
    metrics_per_class = {}
    for i, class_name in enumerate(id2label.values()):
        metrics_per_class.update({
            f'precision_class_{class_name}': precision_per_class[i],
            f'recall_class_{class_name}': recall_per_class[i],
            f'f1_class_{class_name}': f1_per_class[i]
        })
    
    # Confusion matrix - only log to wandb
    cm = confusion_matrix(labels, preds)
    plt.figure(figsize=(10,8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=list(id2label.values()),
                yticklabels=list(id2label.values()))
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
    # Log confusion matrix to wandb
    wandb.log({'eval/confusion_matrix': wandb.Image(plt)})
    plt.close()
    
    # Confidence histograms - only log to wandb
    for i in range(len(id2label)):
        plt.figure(figsize=(8,6))
        class_probs = probs[:, i].numpy()
        plt.hist(class_probs, bins=50)
        plt.title(f'Confidence Distribution - Class {id2label[i]}')
        plt.xlabel('Confidence')
        plt.ylabel('Count')
        wandb.log({f'eval/confidence_dist_class_{id2label[i]}': wandb.Image(plt)})
        plt.close()

    # Log metrics to wandb
    wandb.log({
        'eval/accuracy': accuracy,
        'eval/precision_macro': precision_macro,
        'eval/precision_micro': precision_micro,
        'eval/recall_macro': recall_macro,
        'eval/recall_micro': recall_micro,
        'eval/f1_macro': f1_macro,
        'eval/f1_micro': f1_micro,
    })
    
    # Return only JSON-serializable metrics
    return {
        'accuracy': accuracy,
        'precision_macro': precision_macro,
        'precision_micro': precision_micro,
        'recall_macro': recall_macro,
        'recall_micro': recall_micro,
        'f1_macro': f1_macro,
        'f1_micro': f1_micro,
        **metrics_per_class
    }

class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.gradient_norm = 0.0

    def training_step(self, model, inputs):
        """Override training step to compute training metrics"""
        loss = super().training_step(model, inputs)
        
        # Compute training accuracy
        with torch.no_grad():
            outputs = model(**inputs)
            predictions = outputs.logits.argmax(-1)
            accuracy = (predictions == inputs['labels']).float().mean()
            
            # Log training metrics
            if self.state.global_step % self.args.logging_steps == 0:
                wandb.log({
                    'train/loss': loss.item(),
                    'train/accuracy': accuracy.item(),
                    'train/step': self.state.global_step
                })
        
        # Compute gradient norm
        if self.args.gradient_checkpointing:
            self.gradient_norm = torch.norm(
                torch.stack([
                    torch.norm(p.grad.detach())
                    for p in model.parameters()
                    if p.grad is not None
                ])
            ).item()
        
        return loss

    def log(self, logs: Dict[str, float]) -> None:
        """
        Enhanced logging with additional training metrics
        """
        if self.state.global_step % self.args.logging_steps == 0:
            # Log learning rates
            if hasattr(self.optimizer, "param_groups"):
                current_lr = self.optimizer.param_groups[0]['lr']
                wandb.log({
                    'train/learning_rate': current_lr,
                    'train/step': self.state.global_step
                })
            
            # Log gradient norm
            logs["train/gradient_norm"] = self.gradient_norm
            
            # Log batch size
            logs["train/batch_size"] = self.args.per_device_train_batch_size
            
            # Log memory usage if using GPU
            if torch.cuda.is_available():
                logs["system/gpu_memory_allocated"] = torch.cuda.memory_allocated() / 1024**2  # MB
                logs["system/gpu_memory_cached"] = torch.cuda.memory_reserved() / 1024**2  # MB
            
            # Log parameter statistics
            for name, param in self.model.named_parameters():
                if param.requires_grad:
                    logs[f"parameters/mean/{name}"] = param.data.mean().item()
                    logs[f"parameters/std/{name}"] = param.data.std().item()
                    if param.grad is not None:
                        logs[f"gradients/mean/{name}"] = param.grad.data.mean().item()
                        logs[f"gradients/std/{name}"] = param.grad.data.std().item()
            
            # Log all metrics to wandb
            wandb.log(logs)
        
        super().log(logs)

# Setup Training args
output_dir = "results"

training_args = TrainingArguments(
    output_dir=output_dir,
    report_to='wandb',
    eval_strategy='steps',
    logging_steps=100,
    eval_steps=200,
    save_steps=400,
    save_total_limit=2,

    learning_rate=1e-4,  # LoRA
    warmup_ratio=0.1,
    num_train_epochs=2,
    
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,

    optim="adamw_torch",  # BETTER THAN SGD
    weight_decay=0.01,

    gradient_checkpointing=False,
    dataloader_num_workers=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    run_name="lora_finetuning_run_2",
    
    logging_first_step=True, 
    logging_nan_inf_filter=False, 
    logging_strategy="steps",  
    label_names=["labels"]
)

def get_trainer(model):
    """
    Create a trainer instance with enhanced logging
    """
    # Initialize wandb with detailed config
    wandb.init(
        project="ag_news_classification",
        name=f"roberta_lora_{wandb.util.generate_id()}",
        config={
            "model_name": base_model,
            "lora_config": {
                "r": peft_config.r,
                "alpha": peft_config.lora_alpha,
                "dropout": peft_config.lora_dropout,
                "target_modules": peft_config.target_modules,
            },
            "training_config": {
                "learning_rate": training_args.learning_rate,
                "batch_size": training_args.per_device_train_batch_size,
                "epochs": training_args.num_train_epochs,
                "warmup_ratio": training_args.warmup_ratio,
                "weight_decay": training_args.weight_decay,
            },
            "augmentation_config": augmentation_config,
            "dataset": "ag_news",
            "train_size": len(train_dataset),
            "eval_size": len(eval_dataset),
        }
    )
    
    model.config.label2id = {label: i for i, label in enumerate(class_names)}
    model.config.id2label = {i: label for i, label in enumerate(class_names)}
    
    return CustomTrainer(
        model=model,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
    )

### Start Training

In [20]:
!export CUDA_VISIBLE_DEVICES=3


In [21]:
peft_lora_finetuning_trainer = get_trainer(peft_model)

result = peft_lora_finetuning_trainer.train()

eval/accuracy,▁▁██
eval/f1_class_Business,▁█
eval/f1_class_Sci/Tech,▁█
eval/f1_class_Sports,▁█
eval/f1_class_World,▁█
eval/f1_macro,▁▁██
eval/f1_micro,▁▁██
eval/loss,█▁
eval/precision_class_Business,▁█
eval/precision_class_Sci/Tech,█▁
eval/precision_class_Sports,▁█


/cache_intel/anaconda/miniconda3/lib/python3.9/site-packages/torch/nn/parallel/data_parallel.py:37: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 2 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(
/cache_intel/anaconda/miniconda3/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,Precision Macro,Precision Micro,Recall Macro,Recall Micro,F1 Macro,F1 Micro,Precision Class World,Recall Class World,F1 Class World,Precision Class Sports,Recall Class Sports,F1 Class Sports,Precision Class Business,Recall Class Business,F1 Class Business,Precision Class Sci/tech,Recall Class Sci/tech,F1 Class Sci/tech,Norm,Size,Memory Allocated,Memory Cached,Model.model.roberta.encoder.layer.0.attention.self.query.lora A.default.weight,Model.model.roberta.encoder.layer.0.attention.self.query.lora B.default.weight,Model.model.roberta.encoder.layer.0.attention.self.value.lora A.default.weight,Model.model.roberta.encoder.layer.0.attention.self.value.lora B.default.weight,Model.model.roberta.encoder.layer.1.attention.self.query.lora A.default.weight,Model.model.roberta.encoder.layer.1.attention.self.query.lora B.default.weight,Model.model.roberta.encoder.layer.1.attention.self.value.lora A.default.weight,Model.model.roberta.encoder.layer.1.attention.self.value.lora B.default.weight,Model.model.roberta.encoder.layer.2.attention.self.query.lora A.default.weight,Model.model.roberta.encoder.layer.2.attention.self.query.lora B.default.weight,Model.model.roberta.encoder.layer.2.attention.self.value.lora A.default.weight,Model.model.roberta.encoder.layer.2.attention.self.value.lora B.default.weight,Model.model.roberta.encoder.layer.3.attention.self.query.lora A.default.weight,Model.model.roberta.encoder.layer.3.attention.self.query.lora B.default.weight,Model.model.roberta.encoder.layer.3.attention.self.value.lora A.default.weight,Model.model.roberta.encoder.layer.3.attention.self.value.lora B.default.weight,Model.model.roberta.encoder.layer.4.attention.self.query.lora A.default.weight,Model.model.roberta.encoder.layer.4.attention.self.query.lora B.default.weight,Model.model.roberta.encoder.layer.4.attention.self.value.lora A.default.weight,Model.model.roberta.encoder.layer.4.attention.self.value.lora B.default.weight,Model.model.roberta.encoder.layer.5.attention.self.query.lora A.default.weight,Model.model.roberta.encoder.layer.5.attention.self.query.lora B.default.weight,Model.model.roberta.encoder.layer.5.attention.self.value.lora A.default.weight,Model.model.roberta.encoder.layer.5.attention.self.value.lora B.default.weight,Model.model.roberta.encoder.layer.6.attention.self.query.lora A.default.weight,Model.model.roberta.encoder.layer.6.attention.self.query.lora B.default.weight,Model.model.roberta.encoder.layer.6.attention.self.value.lora A.default.weight,Model.model.roberta.encoder.layer.6.attention.self.value.lora B.default.weight,Model.model.roberta.encoder.layer.7.attention.self.query.lora A.default.weight,Model.model.roberta.encoder.layer.7.attention.self.query.lora B.default.weight,Model.model.roberta.encoder.layer.7.attention.self.value.lora A.default.weight,Model.model.roberta.encoder.layer.7.attention.self.value.lora B.default.weight,Model.model.roberta.encoder.layer.8.attention.self.query.lora A.default.weight,Model.model.roberta.encoder.layer.8.attention.self.query.lora B.default.weight,Model.model.roberta.encoder.layer.8.attention.self.value.lora A.default.weight,Model.model.roberta.encoder.layer.8.attention.self.value.lora B.default.weight,Model.model.roberta.encoder.layer.9.attention.self.query.lora A.default.weight,Model.model.roberta.encoder.layer.9.attention.self.query.lora B.default.weight,Model.model.roberta.encoder.layer.9.attention.self.value.lora A.default.weight,Model.model.roberta.encoder.layer.9.attention.self.value.lora B.default.weight,Model.model.roberta.encoder.layer.10.attention.self.query.lora A.default.weight,Model.model.roberta.encoder.layer.10.attention.self.query.lora B.default.weight,Model.model.roberta.encoder.layer.10.attention.self.value.lora A.default.weight,Model.model.roberta.encoder.layer.10.attention.self.value.lora B.default.weight,Model.model.roberta.encoder.layer.11.attention.self.query.lora A.default.weight,Model.model.roberta.encoder.layer.

/cache_intel/anaconda/miniconda3/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/cache_intel/anaconda/miniconda3/lib/python3.9/site-packages/torch/nn/parallel/data_parallel.py:37: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 2 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(
/cache_intel/anaconda/miniconda3/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/cache_intel/anaconda/miniconda3/lib/python3.9/site-packages/torch/nn/parallel/data_parallel.py:37: UserWa

In [22]:
# Save Model
peft_model_path = os.path.join(output_dir, "peft_model_sentiment")
peft_model.save_pretrained(peft_model_path)
# Save Tokenizer
tokenizer.save_pretrained(peft_model_path)
print(f"Model saved to {peft_model_path}")


Model saved to results/peft_model_sentiment


## Evaluate Finetuned Model


### Run Inference on eval_dataset

In [23]:
from torch.utils.data import DataLoader
import evaluate
from tqdm import tqdm

def evaluate_model(inference_model, dataset, labelled=True, batch_size=32, data_collator=None):
    """
    Evaluate a PEFT model on a dataset.

    Args:
        inference_model: The model to evaluate.
        dataset: The dataset (Hugging Face Dataset) to run inference on.
        labelled (bool): If True, the dataset includes labels and metrics will be computed.
                         If False, only predictions will be returned.
        batch_size (int): Batch size for inference.
        data_collator: Function to collate batches. If None, the default collate_fn is used.

    Returns:
        If labelled is True, returns a tuple (metrics, predictions)
        If labelled is False, returns the predictions.
    """
    # Create the DataLoader
    eval_dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=data_collator)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    inference_model.to(device)
    inference_model.eval()

    all_predictions = []
    if labelled:
        metric = evaluate.load('accuracy')

    # Loop over the DataLoader
    for batch in tqdm(eval_dataloader):
        # Move each tensor in the batch to the device
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = inference_model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        all_predictions.append(predictions.cpu())

        if labelled:
            # Expecting that labels are provided under the "labels" key.
            references = batch["labels"]
            metric.add_batch(
                predictions=predictions.cpu().numpy(),
                references=references.cpu().numpy()
            )

    # Concatenate predictions from all batches
    all_predictions = torch.cat(all_predictions, dim=0)

    if labelled:
        eval_metric = metric.compute()
        print("Evaluation Metric:", eval_metric)
        return eval_metric, all_predictions
    else:
        return all_predictions

In [24]:
# Check evaluation accuracy
_, _ = evaluate_model(peft_model, eval_dataset, True, 32, data_collator)

100%|██████████| 20/20 [00:04<00:00,  4.28it/s]

Evaluation Metric: {'accuracy': 0.9015625}


In [28]:
def process_unlabelled_data(data, tokenizer, augmentations=None):
    """
    专门处理未标记数据的函数
    
    Args:
        data: 包含文本数据的DataFrame/Dataset/list
        tokenizer: 使用的tokenizer
        augmentations: 数据增强配置字典
    
    Returns:
        Dataset: 处理后的数据集
    """
    if augmentations is None:
        augmentations = {
            'synonym': True,
            'delete': True,
            'swap': True,
            'paraphrase': True,
            'noise': True
        }
    
    # 首先打印数据类型和前几条数据来调试
    print(f"Input data type: {type(data)}")
    print("First few items:")
    print(data.head() if hasattr(data, 'head') else data[:5])
    
    # 获取文本列表
    texts = []
    if isinstance(data, pd.DataFrame):
        if 'text' in data.columns:
            texts = data['text'].tolist()
        else:
            print("Available columns:", data.columns)
            raise ValueError("No 'text' column found in DataFrame")
    elif isinstance(data, (list, np.ndarray)):
        texts = data
    else:
        try:
            texts = list(data)
        except:
            raise ValueError(f"Unsupported data type: {type(data)}")
    
    # 确保所有文本都是字符串
    texts = [str(t) if t is not None else "" for t in texts]
    
    # 清理和增强文本
    processed_texts = []
    for text in tqdm(texts, desc="Processing texts"):
        # 清理文本
        try:
            clean = clean_text(text)
        except Exception as e:
            print(f"Error cleaning text: {text}")
            print(f"Error message: {str(e)}")
            clean = text  # 如果清理失败，使用原始文本
        
        # 应用数据增强
        aug_text = clean
        if augmentations.get('synonym', False):
            try:
                aug_text = synonym_aug.augment(aug_text)[0]
            except:
                pass
        if augmentations.get('delete', False):
            try:
                aug_text = delete_aug.augment(aug_text)[0]
            except:
                pass
        if augmentations.get('swap', False):
            try:
                aug_text = swap_aug.augment(aug_text)[0]
            except:
                pass
        if augmentations.get('paraphrase', False):
            aug_text = simple_paraphrase(aug_text)
        if augmentations.get('noise', False):
            try:
                aug_text = typo_aug.augment(aug_text)[0]
            except:
                pass
        
        processed_texts.append(aug_text)
    
    # 批量tokenize
    tokenized = tokenizer(
        processed_texts,
        truncation=True,
        padding=True,
        max_length=256,
        return_tensors="pt"
    )
    
    # 转换为Dataset格式
    dataset = Dataset.from_dict({
        'input_ids': tokenized['input_ids'],
        'attention_mask': tokenized['attention_mask']
    })
    
    return dataset

# 使用示例：
# 读取未标记数据
unlabelled_dataset = pd.read_pickle("/home/viewsetting/ssd_2T/test_unlabelled.pkl")

# 让我们先看看数据的结构
print("Dataset info:")
print(type(unlabelled_dataset))
if isinstance(unlabelled_dataset, pd.DataFrame):
    print("\nColumns:", unlabelled_dataset.columns)
    print("\nFirst few rows:")
    print(unlabelled_dataset.head())

# 处理数据
test_dataset = process_unlabelled_data(
    unlabelled_dataset,
    tokenizer,
    augmentations={
        'synonym': True,
        'delete': True,
        'swap': True,
        'paraphrase': True,
        'noise': True
    }
)

# 使用示例：
# 读取未标记数据
unlabelled_dataset = pd.read_pickle("/home/viewsetting/ssd_2T/test_unlabelled.pkl")

# 处理数据
test_dataset = process_unlabelled_data(
    unlabelled_dataset,
    tokenizer,
    augmentations={
        'synonym': True,
        'delete': True,
        'swap': True,
        'paraphrase': True,
        'noise': True
    }
)

# 运行推理
preds = evaluate_model(peft_model, test_dataset, labelled=False, batch_size=32, data_collator=data_collator)

# 保存结果
df_output = pd.DataFrame({
    'ID': range(len(preds)),
    'Label': preds.numpy()
})
df_output.to_csv(os.path.join(output_dir, "inference_output_sentiment.csv"), index=False)
print("Inference complete. Predictions saved to inference_output_sentiment.csv")

Dataset info:
<class 'datasets.arrow_dataset.Dataset'>
Input data type: <class 'datasets.arrow_dataset.Dataset'>
First few items:
{'text': ['Remains of New Species of Hobbit-Sized Human Found Scientists in Australia have found a new species of hobbit-sized humans who lived about 18,000 years ago on an Indonesian island in a discovery that adds another piece to the complex puzzle of human evolution.', 'Iran to cease negotiations with EU in case of dead end A top Iranian official said Sunday that Iran would withdraw from the negotiations with the European Union (EU) if the upcoming talks in Brussels turned into a dead-end, the official IRNA news agency reported.', 'Israel levels new accusations against Syria Without acknowledging responsibility for the car-bombing death of a Hamas activist in Syria, Israeli Deputy Defense Minister Zeev Boim yesterday issued a toughly worded ', 'Enevo a Silicon Valley startup create self-powered battery and another new company building project creating lo

Processing texts: 100%|██████████| 8000/8000 [00:29<00:00, 267.00it/s]


Input data type: <class 'datasets.arrow_dataset.Dataset'>
First few items:
{'text': ['Remains of New Species of Hobbit-Sized Human Found Scientists in Australia have found a new species of hobbit-sized humans who lived about 18,000 years ago on an Indonesian island in a discovery that adds another piece to the complex puzzle of human evolution.', 'Iran to cease negotiations with EU in case of dead end A top Iranian official said Sunday that Iran would withdraw from the negotiations with the European Union (EU) if the upcoming talks in Brussels turned into a dead-end, the official IRNA news agency reported.', 'Israel levels new accusations against Syria Without acknowledging responsibility for the car-bombing death of a Hamas activist in Syria, Israeli Deputy Defense Minister Zeev Boim yesterday issued a toughly worded ', 'Enevo a Silicon Valley startup create self-powered battery and another new company building project creating low-C systems so is probably said in reality also include

100%|██████████| 250/250 [00:28<00:00,  8.72it/s]

Inference complete. Predictions saved to inference_output_sentiment.csv
